In [2]:
import pandas as pd
import param as pm
import panel as pn
import hvplot.pandas
import holoviews as hv
import numpy as np

pn.extension()

In [106]:
class Fundraising(pm.Parameterized):
    total_raised = pm.Number(1e6, bounds=(1e5, 1e7), precedence=-1)
    hatch_tribute = pm.Number(0.05, bounds=(0,1), precedence=-1)
    hatch_price = pm.Number(1, bounds=(0.5,2), precedence=-1)
    initial_price = pm.Number(2, bounds=(1,4))
    expected_growth = pm.Number(2, bounds=(1,100))
    reserve_ratio = pm.Number(0.1, bounds=(0,1))
    entry_tribute = pm.Number(0.0, bounds=(0,0.5), precedence=-1)
    exit_tribute = pm.Number(0.0, bounds=(0,0.5), precedence=-1)
    token_supply = pm.Number(1e6)
    reserve_balance = pm.Number(1e6)
    
    
    def xrate(self):
        return 1 / self.hatch_price
    
    def growth(self):
        return self.expected_growth
    
    def pctOffered(self):
        return 1
    
    def pctBeneficiary(self):
        return self.hatch_tribute
    
    def sSupply(self):
        return self.total_raised * self.xrate()
    
    def sBalance(self):
        return self.total_raised * (1 - self.hatch_tribute)
    
    def sPrice(self):
        return self.initial_price
    
    def sMarketCap(self):
        return self.sSupply() * self.sPrice()
    
    def eMarketCap(self):
        return self.sMarketCap() * self.growth()
    
    def ePrice(self):
        return self.sPrice() * np.sqrt(self.growth())
    
    def ppSupply(self):
#         ppSupply = (ePrice / (eMarketCap ** (1 - reserveRatio) * (sPrice ** reserveRatio))) ** (1 / (reserveRatio - 1))
        return (self.ePrice() / (self.eMarketCap()**(1 - self.reserve_ratio) * (self.sPrice() ** self.reserve_ratio))) ** (1 / (self.reserve_ratio - 1))
                                                   
    def vSupply(self):
        return self.ppSupply() - self.sSupply()
    
    def ppBalance(self):
        return self.sPrice() * self.ppSupply() * self.reserve_ratio
    
    def vBalance(self):
        return self.ppBalance() - self.sBalance()
    
    def get_outputs(self):
        outputs = pd.DataFrame({
            "sSupply": f.sSupply(),
            "sBalance": f.sBalance(),
            "sPrice": f.sPrice(),
            "sMarketcap": f.sMarketCap(),
            "ePrice": f.ePrice(),
            "eMarketCap": f.eMarketCap(),
            "ppSupply": f.ppSupply(),
            "vSupply": f.vSupply(),
            "ppBalance": f.ppBalance(),
            "vBalance": f.vBalance(),
        }, index=['Value'])
        return outputs
    
    def view_outputs(self):
        outputs = self.get_outputs()
        return outputs.apply(lambda x: round(x, 2)).T

    def buy_amount(self, supply, collateral, pay_amount, reserve_ratio):
        # buyAmt = tokenSupply * ((1 + amtPaid / collateral)^CW — 1)
        return supply * ((1 + pay_amount / collateral)**reserve_ratio - 1)
    
    def sell_amount(self, supply, collateral, sell_amount, reserve_ratio):
        # sellAmt = collateral * ((1 + tokensSold / totalSupply)^(1/CW) — 1)
        return collateral * ((1 + sell_amount / supply)**(1/reserve_ratio) - 1)
    
    def make_buy_order(self, buy_amount):
        fee = buy_amount * self.entry_tribute
        buy_amount = buy_amount - fee
        self.reserve_balance += buy_amount
        collateral_supply = self.token_supply + self.vSupply()
        collateral_balance = self.reserve_balance + self.vBalance()
        return_amount = self.buy_amount(collateral_supply, collateral_balance, buy_amount, self.reserve_ratio)
        self.token_supply += return_amount
        return return_amount
        
    def make_sell_order(self, sell_amount):
        collateral_supply = self.token_supply + self.vSupply()
        collateral_balance = self.reserve_balance + self.vBalance()
        return_amount = self.sell_amount(collateral_supply, collateral_balance, sell_amount, self.reserve_ratio)
        fee = return_amount * self.exit_tribute 
        self.token_supply -= sell_amount
        self.reserve_balance -= return_amount - fee
        return return_amount - fee        

In [107]:
f = Fundraising()

In [108]:
pn.Row(f, f.view_outputs)

Row
    [0] Column(margin=5, name='Fundraising', width=300)
        [0] StaticText(value='<b>Fundraising</b>')
        [1] FloatSlider(end=4, name='Initial price', start=1, value=2)
        [2] FloatSlider(end=100, name='Expected growth', start=1, value=2)
        [3] FloatSlider(name='Reserve ratio', value=0.1)
        [4] FloatInput(name='Token supply', value=1000000.0)
        [5] FloatInput(name='Reserve balance', value=1000000.0)
    [1] ParamMethod(method)

In [114]:
f.make_buy_order(1000)

45.558339435138045

In [ ]:
class Fundraising(pm.Parameterized):
    total_raised = pm.Number(1e6, bounds=(1e5, 1e7), precedence=-1)
    hatch_tribute = pm.Number(0.05, bounds=(0,1), precedence=-1)
    hatch_price = pm.Number(1, bounds=(0.5,2), precedence=-1)
    initial_price = pm.Number(2, bounds=(1,4))
    expected_growth = pm.Number(2, bounds=(1,100))
    reserve_ratio = pm.Number(0.1, bounds=(0,1))
    entry_tribute = pm.Number(0.05, bounds=(0,0.5), precedence=-1)
    exit_tribute = pm.Number(0.05, bounds=(0,0.5), precedence=-1)
    
    
    def sBalance(self):
        return self.total_raised * (1 - self.hatch_tribute)
    
    def sMarketCap(self):
        return self.initial_price * self.total_raised / self.hatch_price
    
    def eMarketCap(self):
        return self.expected_growth * self.sMarketCap()
    
    def ePrice(self):
        return self.initial_price * np.sqrt(self.expected_growth)
    
    def ppSupply(self):
        return (self.ePrice() / (self.eMarketCap()**(1 - self.reserve_ratio) * (self.initial_price ** self.reserve_ratio))) ** (1 / (self.reserve_ratio - 1))
                                                   
    def vSupply(self):
        return self.ppSupply() - (self.total_raised / self.hatch_price)
    
    def ppBalance(self):
        return self.initial_price * self.ppSupply() * self.reserve_ratio
    
    def vBalance(self):
        return self.ppBalance() - self.sBalance()
    
    def get_outputs(self):
        outputs = pd.DataFrame({
            "sSupply": f.sSupply(),
            "sBalance": f.sBalance(),
            "sPrice": f.sPrice(),
            "sMarketcap": f.sMarketCap(),
            "ePrice": f.ePrice(),
            "eMarketCap": f.eMarketCap(),
            "ppSupply": f.ppSupply(),
            "vSupply": f.vSupply(),
            "ppBalance": f.ppBalance(),
            "vBalance": f.vBalance(),
        }, index=['Value'])
        return outputs
    
    def view_outputs(self):
        outputs = self.get_outputs()
        return outputs.apply(lambda x: round(x, 2)).T